# Selenium para descargar base de dato de la intranet de Opción de Energía

## Librerías

In [38]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
import pandas as pd
import glob
import os

## Page 1 - Login Opcion Comercializadora Intranet

In [176]:
driver = webdriver.Chrome("../Programas/chromedriver")

In [177]:
url = "https://canalagentes.opcionenergia.com/authentication/login"

In [178]:
driver.get(url)

In [179]:
driver.find_element_by_name("user").send_keys("vivolt")
driver.find_element_by_name("password").send_keys("temporal01vivolt")
driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)

## Page 2 - Download "SIPS"

Para asignar la variable busco por el xpath, y desde tools de chrome, abro el codigo html y copio el xpath. 

Pulsando el boton de SIPS:

In [180]:
sips = driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li')

In [181]:
sips.click()

rellenando el CUPS: # agregar funcion para rellenar el cups desde la API

In [182]:
driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")

In [183]:
mostrar_grafico_sips = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button')

In [184]:
mostrar_grafico_sips.click()

In [185]:
descargar_csv = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button')

In [186]:
descargar_csv.click()

## Funciones para el proceso

In [149]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../configuration')
from config import login_data, client_cups

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#send keys con valores escondidos en config. Por ahora datos insertados de forma manual. 

In [198]:
def download_consumo(login_data, client_cups):
    driver = webdriver.Chrome("../Programas/chromedriver")
    driver.get(url)
    sleep(1.5)
    driver.find_element_by_name("user").send_keys("vivolt")
    driver.find_element_by_name("password").send_keys("temporal01vivolt")
    sleep(0.5)
    driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)
    sleep(1.5)
    driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li').click()
    sleep(0.5)
    driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")
    sleep(0.5)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button').click()
    sleep(0.5)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button').click()

In [199]:
download_consumo(login_data, client_cups)

## Import CSV

In [197]:
def import_ConsumoCliente():
    list_of_files = glob.glob('/Users/pptrv/Downloads/*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    data = pd.read_csv(latest_file, error_bad_lines=False, sep=",")
    return(data)

In [147]:
import_ConsumoCliente()

b'Skipping line 4: expected 1 fields, saw 2\nSkipping line 5: expected 1 fields, saw 2\nSkipping line 6: expected 1 fields, saw 2\nSkipping line 93: expected 1 fields, saw 7\nSkipping line 94: expected 1 fields, saw 7\nSkipping line 95: expected 1 fields, saw 7\nSkipping line 96: expected 1 fields, saw 7\nSkipping line 97: expected 1 fields, saw 7\nSkipping line 98: expected 1 fields, saw 7\nSkipping line 99: expected 1 fields, saw 7\nSkipping line 100: expected 1 fields, saw 7\nSkipping line 101: expected 1 fields, saw 7\nSkipping line 102: expected 1 fields, saw 7\nSkipping line 103: expected 1 fields, saw 7\nSkipping line 104: expected 1 fields, saw 7\nSkipping line 105: expected 1 fields, saw 7\nSkipping line 106: expected 1 fields, saw 7\nSkipping line 107: expected 1 fields, saw 7\nSkipping line 108: expected 1 fields, saw 7\nSkipping line 109: expected 1 fields, saw 7\nSkipping line 110: expected 1 fields, saw 7\nSkipping line 111: expected 1 fields, saw 7\nSkipping line 112: ex

,CUPS;;ES0021000005204736CV
0,Tarifa;;3.0A
1,Potencia Contratada (kWh);;P4;0
2,Potencia Contratada (kWh);;P5;0
3,Potencia Contratada (kWh);;P6;0
4,Tension;;3X400/230
...,...
87,P2;;7825
88,P3;;738
89,P4;;156
90,P5;;457


## Cleaning DF

## Saving DF to CSV in Final_Proyect_Vivolt/Data/CSV

In [ ]:
consumo_cliente_df.to_csv("../Data/csv/consumo_cliente.csv")